# CDF Response Time Analysis for Open Network with Two Classes

This example demonstrates CDF response time analysis for an open network with two classes and class switching. It compares results from Fluid solver and JMT transient analysis.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.fluid.*
import jline.solvers.jmt.*
import jline.util.matrix.*

In [ ]:
// CDF Response Time Analysis Example 3
// Open network with two classes, comparing Fluid and JMT transient CDF

val model = Network("myModel")

// Block 1: nodes
val source = Source(model, "Source")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)
val sink = Sink(model, "Sink")

// Block 2: classes
val class1 = OpenClass(model, "Class1", 0)
val class2 = OpenClass(model, "Class2", 0)

In [ ]:
// Arrival processes
source.setArrival(class1, Exp.fitMean(4.0))  // Source,Class1
source.setArrival(class2, Exp.fitMean(4.0))  // Source,Class2

// Service processes
queue1.setService(class1, Exp.fitMean(1.0))  // Queue1,Class1
queue1.setService(class2, Exp.fitMean(1.0))  // Queue1,Class2

queue2.setService(class1, Exp.fitMean(1.0))  // Queue2,Class1
queue2.setService(class2, Exp.fitMean(1.0))  // Queue2,Class2

In [ ]:
// Block 3: topology - complex routing matrix with class switching
val P = model.initRoutingMatrix()

// Class 1 routing
P.set(class1, class1, source, queue1, 1.0)  // Source->Queue1
P.set(class1, class2, queue1, queue2, 1.0)  // Queue1->Queue2 (class switch)
P.set(class2, class1, queue2, sink, 1.0)    // Queue2->Sink

// Class 2 routing  
P.set(class2, class2, source, queue1, 1.0)  // Source->Queue1
P.set(class2, class1, queue1, queue2, 1.0)  // Queue1->Queue2 (class switch)
P.set(class1, class2, queue2, sink, 1.0)    // Queue2->Sink

model.link(P)

In [ ]:
// Solve with Fluid solver
val fluidOptions = FLD.defaultOptions()
fluidOptions.iterMax = 300

val fluidSolver = FLD(model, fluidOptions)
val rdFluid = fluidSolver.cdfRespT
println("RDfluid obtained")

In [ ]:
// Solve with JMT transient CDF analysis
val jmtOptions = JMT.defaultOptions()
jmtOptions.samples = 10000
jmtOptions.seed = 23000

val jmtSolver = JMT(model, jmtOptions)
val rdSim = jmtSolver.cdfRespT
println("RDsim obtained")

In [ ]:
// Display results comparison for queue stations only (stations 1 and 2)
println("\n=== CDF Response Time Analysis Results (Queue Stations Only) ===")

for (i in 1 until model.numberOfStations - 1) {  // Only queue stations (1, 2)
    println("\nNode $i (${model.stations[i].name}):")
    
    for (c in 0 until model.numberOfClasses) {
        val className = model.classes[c].name
        println("  Class $c ($className):")
        
        // Display fluid results sample
        if (rdFluid[i][c] != null) {
            val fluidData = rdFluid[i][c]
            println("    Fluid CDF points: ${fluidData.length()}")
            if (fluidData.length() > 0) {
                println("    Fluid range: [${fluidData.get(0, 1)}, ${fluidData.get(fluidData.length()-1, 1)}]")
                // Show tail probability at different percentiles
                val n = fluidData.length()
                if (n > 10) {
                    val idx90 = (n * 0.9).toInt()
                    val tail90 = 1.0 - fluidData.get(idx90, 0)
                    println("    Fluid tail (90th percentile): $tail90")
                }
            }
        }
        
        // Display simulation results sample
        if (rdSim[i][c] != null) {
            val simData = rdSim[i][c]
            println("    Simulation CDF points: ${simData.length()}")
            if (simData.length() > 0) {
                println("    Simulation range: [${simData.get(0, 1)}, ${simData.get(simData.length()-1, 1)}]")
                // Show tail probability at different percentiles
                val n = simData.length()
                if (n > 10) {
                    val idx90 = (n * 0.9).toInt()
                    val tail90 = 1.0 - simData.get(idx90, 0)
                    println("    Simulation tail (90th percentile): $tail90")
                }
            }
        }
    }
}

In [ ]:
// Calculate average response times from CDFs (queue stations only)
val avgRespTFromCDFFluid = Matrix.zeros(model.numberOfStations, model.numberOfClasses)
val avgRespTFromCDFSim = Matrix.zeros(model.numberOfStations, model.numberOfClasses)

for (i in 1 until model.numberOfStations - 1) {  // Only queue stations
    for (c in 0 until model.numberOfClasses) {
        // From Fluid CDF
        if (rdFluid[i][c] != null && rdFluid[i][c].length() > 1) {
            val fluidData = rdFluid[i][c]
            var sum = 0.0
            for (j in 1 until fluidData.length()) {
                val diff = fluidData.get(j, 0) - fluidData.get(j-1, 0)
                val value = fluidData.get(j, 1)
                sum += diff * value
            }
            avgRespTFromCDFFluid.set(i, c, sum)
        }
        
        // From Simulation CDF
        if (rdSim[i][c] != null && rdSim[i][c].length() > 1) {
            val simData = rdSim[i][c]
            var sum = 0.0
            for (j in 1 until simData.length()) {
                val diff = simData.get(j, 0) - simData.get(j-1, 0)
                val value = simData.get(j, 1)
                sum += diff * value
            }
            avgRespTFromCDFSim.set(i, c, sum)
        }
    }
}

println("\nAverage Response Times from CDF (Fluid):")
avgRespTFromCDFFluid.print()
println("\nAverage Response Times from CDF (Simulation):")
avgRespTFromCDFSim.print()

This example demonstrates:

1. **Open Network with Class Switching**: 
   - Two classes that switch between each other as they traverse the network
   - Complex routing matrix with class transitions
2. **Multiple Solution Methods**:
   - Fluid solver for analytical steady-state CDF analysis
   - JMT simulation for comparison and validation
3. **Response Time Analysis**:
   - Focus on queue stations (excluding source and sink)
   - Computation of tail probabilities and average response times
4. **Class Switching Dynamics**:
   - Class 1 jobs switch to Class 2 after Queue1
   - Class 2 jobs switch to Class 1 after Queue1

The class switching creates complex traffic patterns that can be analyzed using CDF methods to understand response time distributions beyond just mean values.